## Animated visualizing dynamic of commits per user in project1

In [7]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import numpy as np

In [8]:
con = sqlite3.connect("../datasets/checking-logs.sqlite")

In [9]:
sql = """
SELECT uid, DATE(timestamp) as date, COUNT(*) AS commits
FROM checker
WHERE labname == 'project1' AND status == 'ready' AND uid LIKE 'user_%'
GROUP BY uid, date
ORDER By date
"""
df = pd.read_sql(sql,con, parse_dates='date')
df['cum_commits'] = df.groupby('uid')['commits'].cumsum()
del df['commits']
zero_points = df.groupby('uid').first().reset_index()

zero_points['cum_commits'] = 0
zero_points['date'] = zero_points['date'] - pd.DateOffset(days=1) 
last_points = df.groupby('uid').last().reset_index()
last_points['date'] = last_points['date'].max()
df = pd.concat([zero_points, df, last_points]).sort_values(['date', 'uid'])


In [10]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

users = df['uid'].unique()

date_mapping = {date: i for i, date in enumerate(sorted(df['date'].unique()))}
df['day_num'] = df['date'].map(date_mapping)
all_dates = df['day_num'].unique()
numFrames = len(all_dates)
DARKGRID_COLORS = {
    'background': '#fff0f0',  # #EAEAF2
    'grid': '#FFFFFF',
}


initial_data = []
for user in users:
    user_data = df[df['uid'] == user]
    initial_point = user_data[user_data['day_num'] == all_dates[0]]
    initial_data.append(
        go.Scatter(
            x=[all_dates[0]],
            y=initial_point['cum_commits'],
            mode='lines+markers', 
            name=user,
            line=dict(width=2)
        )
    )

frames = []
for i in range(1, numFrames):
    current_date = all_dates[i]
    frame_data = []
    for user in users:
        user_data = df[(df['uid'] == user) & (df['day_num'] <= current_date)]
        if not user_data.empty:
            frame_data.append(
                go.Scatter(
                    x=user_data['day_num'],
                    y=user_data['cum_commits'],
                    mode='lines+markers',
                    name=user,
                    line=dict(width=2)
                )
            )
    frames.append(go.Frame(data=frame_data))


fig = go.Figure(
    data=initial_data,
    layout=go.Layout(
        title=dict(
            text='Dynamic of commits per user in project1',
        ),
        xaxis=dict(
            range=[1,21.5],
            dtick=2,
        ),
        yaxis=dict(
            range=[0, 170],
            dtick=20
        ),
        paper_bgcolor=DARKGRID_COLORS['background'],
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(
                label="Play",
                method="animate",
                args=[None, {"frame": {"duration": 500, "redraw": True}}]
            )]
        ),]
    ),
    frames=frames
)

fig.show()

In [11]:
con.close()